In [ ]:
from haystack.nodes import PDFToTextConverter
from haystack.nodes import PreProcessor
from haystack.document_stores import PineconeDocumentStore
from dotenv import dotenv_values
from pathlib import Path
from haystack.pipelines import Pipeline
import pandas as pd
import numpy as np
import openai
import pinecone
import os

In [ ]:
config = dotenv_values('api_keys.env')
openai.api_key = config['OPENAI_API_KEY']
pinecone_api_key = config['PINECONE_API_KEY']
pinecone_env = config['PINECONE_ENV']

In [ ]:
pinecone.init(api_key=pinecone_api_key,
              environment=pinecone_env)

# pinecone.create_index(name='diw-test-index',
#                        dimension=1536)
pinecone.list_indexes()

In [ ]:
document_store = PineconeDocumentStore(
    api_key=pinecone_api_key,
    environment=pinecone_env,
    similarity="cosine",
    index='testing',
    embedding_dim=1536
)

In [ ]:
df = pd.read_pickle('diw.pkl')
print(df.head())

In [ ]:
meta_data = df.to_dict('records')

In [ ]:

converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["de","en"]
)

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0
)

# document_store = InMemoryDocumentStore(use_bm25=True)




In [ ]:
def get_embeddings(text: str, model="text-embedding-ada-002") -> list[float]:
    embedding = openai.Embedding.create(
        input=text,
        model=model
    )["data"][0]["embedding"]

    return embedding

In [ ]:
def get_embeddings_for_pinecone(text: str, model="text-embedding-ada-002") -> list[str]:
    embedding = openai.Embedding.create(
        input=text,
        model=model
    )["data"][0]["embedding"]

    return [str(i) for i in embedding]

In [ ]:
input_string = 'I am the terminator'

x = get_embeddings_for_pinecone(text=input_string)
print(type(x[0]))

In [ ]:
def create_prompt(dataframe, df_column, question):

    content = []
    dataframe[df_column].apply(lambda x: content.append(str(x).replace(",", '').replace("'", '')))

    header = """Answer the question as truthfully as possible using the provided context, and if the answer
     is not contained within the text below, say "I don't know."\n\nContext:\n"""

    header_total = str(header + str(content)).replace('[', '').replace(']', '')

    return header_total + "\n\n Q: " + question + "\n A:"

In [ ]:
for count, filename in enumerate(os.listdir("pdfs/")):
    if count < 1:
        print(f'Working on {filename}')
        doc = converter.convert(file_path=Path("pdfs/"+str(filename)), meta=meta_data[count])
        processed_docs = processor.process(doc)
        
        for doc_count in range(len(processed_docs)):
            embedding_meta_data = get_embeddings_for_pinecone(text=processed_docs[doc_count].content)
            
            processed_docs[doc_count].meta['embedding'] = embedding_meta_data

        document_store.write_documents(processed_docs)
    else:
        break

In [ ]:
index = pinecone.Index('testing')

In [ ]:
index.describe_index_stats()

In [ ]:
index.fetch(ids=['id1'])